In [ ]:
import os
import numpy as np
import cv2
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, AveragePooling2D, Flatten, Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint

In [ ]:
# Define the paths to the train folder containing real and fake subfolders
train_real_path = "/content/drive/MyDrive/Validation/Real"
train_fake_path = "/content/drive/MyDrive/Validation/Fake"

In [ ]:
# Define the number of real and fake images you want to use for training
num_real_images = 19000
num_fake_images = 19000

In [ ]:
# Function to load and preprocess images
def load_images_from_folder(folder_path, num_images):
    images = []
    count = 0
    for filename in os.listdir(folder_path):
        if count >= num_images:
            break
        img_path = os.path.join(folder_path, filename)
        img = cv2.imread(img_path)
        img = cv2.resize(img, (64, 64))  # Resizing images to fit LeNet architecture
        images.append(img)
        count += 1
    return np.array(images)

In [ ]:
# Load real and fake images
real_images = load_images_from_folder(train_real_path, num_real_images)
fake_images = load_images_from_folder(train_fake_path, num_fake_images)

In [ ]:
# Create labels for real and fake images
real_labels = np.ones((num_real_images,))
fake_labels = np.zeros((num_fake_images,))

In [ ]:
# Concatenate real and fake images and labels
X_train = np.concatenate((real_images, fake_images), axis=0)
y_train = np.concatenate((real_labels, fake_labels), axis=0)

In [ ]:
# Shuffle the data
shuffle_indices = np.arange(X_train.shape[0])
np.random.shuffle(shuffle_indices)
X_train = X_train[shuffle_indices]
y_train = y_train[shuffle_indices]


In [ ]:
# Normalize pixel values to range [0, 1]
X_train = X_train / 255.0

In [ ]:
from tensorflow.keras.layers import LeakyReLU
from tensorflow.keras import regularizers

# Define the L2 regularization parameter
l2_reg = 0.001  # You can adjust this parameter

model = Sequential([
    Conv2D(6, kernel_size=(5, 5), input_shape=(64, 64, 3),
           kernel_regularizer=regularizers.l2(l2_reg)),
    LeakyReLU(alpha=0.1),
    AveragePooling2D(pool_size=(2, 2)),
    Conv2D(16, kernel_size=(5, 5),
           kernel_regularizer=regularizers.l2(l2_reg)),
    LeakyReLU(alpha=0.1),
    AveragePooling2D(pool_size=(2, 2)),
    Flatten(),
    Dense(120, kernel_regularizer=regularizers.l2(l2_reg)),
    LeakyReLU(alpha=0.1),
    Dense(84, kernel_regularizer=regularizers.l2(l2_reg)),
    LeakyReLU(alpha=0.1),
    Dense(1, activation='sigmoid')
])

In [ ]:
# Compile the model
model.compile(loss='binary_crossentropy',
              optimizer=Adam(),
              metrics=['accuracy'])

In [ ]:
# Define model checkpoints
checkpoint_path = "lenet_deepfake_detection.h5"
checkpoint = ModelCheckpoint(checkpoint_path, monitor='val_accuracy', verbose=1, save_best_only=True, mode='max')


In [ ]:
# Define early stopping
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
early_stopping = EarlyStopping(monitor='val_accuracy', patience=10, verbose=1, mode='max', restore_best_weights=True)


In [ ]:
# Train the model with early stopping
history = model.fit(X_train, y_train,
                    batch_size=32,
                    epochs=100,  # You can increase the number of epochs
                    validation_split=0.2,
                    callbacks=[checkpoint, early_stopping])

Epoch 1/100
950/950 [==============================] - ETA: 0s - loss: 0.5897 - accuracy: 0.7429
Epoch 1: val_accuracy improved from -inf to 0.77658, saving model to lenet_deepfake_detection.h5
950/950 [==============================] - 96s 100ms/step - loss: 0.5897 - accuracy: 0.7429 - val_loss: 0.5158 - val_accuracy: 0.7766
Epoch 2/100


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


950/950 [==============================] - ETA: 0s - loss: 0.4954 - accuracy: 0.7907
Epoch 2: val_accuracy improved from 0.77658 to 0.79605, saving model to lenet_deepfake_detection.h5
950/950 [==============================] - 92s 97ms/step - loss: 0.4954 - accuracy: 0.7907 - val_loss: 0.4853 - val_accuracy: 0.7961
Epoch 3/100
950/950 [==============================] - ETA: 0s - loss: 0.4587 - accuracy: 0.8115
Epoch 3: val_accuracy did not improve from 0.79605
950/950 [==============================] - 97s 102ms/step - loss: 0.4587 - accuracy: 0.8115 - val_loss: 0.4889 - val_accuracy: 0.7905
Epoch 4/100
950/950 [==============================] - ETA: 0s - loss: 0.4353 - accuracy: 0.8255
Epoch 4: val_accuracy improved from 0.79605 to 0.82803, saving model to lenet_deepfake_detection.h5
950/950 [==============================] - 94s 99ms/step - loss: 0.4353 - accuracy: 0.8255 - val_loss: 0.4300 - val_accuracy: 0.8280
Epoch 5/100
950/950 [==============================] - ETA: 0s - loss:

In [ ]:
test_real_path = "/content/drive/MyDrive/Test/Real"
test_fake_path = "/content/drive/MyDrive/Test/Fake"


In [ ]:
# Define the number of real and fake images you want to use for testing
num_test_real_images = 1000
num_test_fake_images = 1000

In [ ]:
# Function to load and preprocess test images
def load_test_images_from_folder(folder_path, num_images):
    images = []
    count = 0
    for filename in os.listdir(folder_path):
        if count >= num_images:
            break
        img_path = os.path.join(folder_path, filename)
        img = cv2.imread(img_path)
        img = cv2.resize(img, (64, 64))  # Resizing images to fit LeNet architecture
        images.append(img)
        count += 1
    return np.array(images)


In [ ]:
# Load test real and fake images
test_real_images = load_test_images_from_folder(test_real_path, num_test_real_images)
test_fake_images = load_test_images_from_folder(test_fake_path, num_test_fake_images)


In [ ]:
# Create labels for test real and fake images
test_real_labels = np.ones((num_test_real_images,))
test_fake_labels = np.zeros((num_test_fake_images,))

In [ ]:

# Concatenate test real and fake images and labels
X_test = np.concatenate((test_real_images, test_fake_images), axis=0)
y_test = np.concatenate((test_real_labels, test_fake_labels), axis=0)

In [ ]:

# Shuffle the test data
shuffle_indices_test = np.arange(X_test.shape[0])
np.random.shuffle(shuffle_indices_test)
X_test = X_test[shuffle_indices_test]
y_test = y_test[shuffle_indices_test]

In [ ]:

# Normalize pixel values to range [0, 1]
X_test = X_test / 255.0

In [ ]:

# Evaluate the model on the test data
loss, accuracy = model.evaluate(X_test, y_test, verbose=1)
print("Test Loss:", loss)
print("Test Accuracy:", accuracy)

63/63 [==============================] - 2s 32ms/step - loss: 0.3233 - accuracy: 0.9155
Test Loss: 0.32334011793136597
Test Accuracy: 0.9154999852180481


In [ ]:
loss, accuracy = model.evaluate(X_train[:600], y_train[:600], verbose=1)
print("Test Loss:", loss)
print("Test Accuracy:", accuracy)

19/19 [==============================] - 1s 31ms/step - loss: 0.2682 - accuracy: 0.9417
Test Loss: 0.2682012617588043
Test Accuracy: 0.9416666626930237
